<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase3/blob/main/Fase3_TechChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning do Modelo BERT com AmazonTitles-1.3MM

Neste notebook, realizaremos o fine-tuning do modelo BERT (`bert-base-uncased`) usando o dataset "The AmazonTitles-1.3MM". O objetivo é treinar o modelo para que ele consiga gerar descrições de produtos com base em seus títulos.

### 1. Instalar dependências


In [11]:
# Instalar as bibliotecas necessárias
!pip install datasets transformers torch pandas 'transformers[torch]' gdown

## 2. Importar as Bibliotecas e preparar o Ambiente

In [12]:
import json
import torch
import gzip
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Verificar se temos acesso a uma GPU no Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


## 3. Carregar o Tokenizer e o Modelo BERT

In [13]:
# Carregar o tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Carregar o modelo BERT para classificação
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 4. Classe Dataset para Gerenciamento de Dados

In [14]:
class AmazonTitlesDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


##5. Leitura em Chunks e Tokenização (para JSONL compactado em GZIP)
Vamos ajustar a função de leitura para processar arquivos JSONL. Cada linha é um objeto JSON separado, então a função simplesmente lê uma linha por vez e cria blocos (chunks).

In [15]:
def ler_arquivo_em_blocos_jsonl_gz(caminho_arquivo, tamanho_bloco=10000):
    with gzip.open(caminho_arquivo, 'rt') as f:  # 'rt' para ler como texto
        bloco = []
        for i, linha in enumerate(f):
            bloco.append(json.loads(linha.strip()))  # Lê uma linha como JSON
            if (i + 1) % tamanho_bloco == 0:
                yield bloco
                bloco = []
        if bloco:
            yield bloco

def processar_e_tokenizar_chunk(chunk, max_length=128):
    titles = [item['title'] for item in chunk]
    descriptions = [item['content'] for item in chunk]

    # Concatenar título e descrição
    inputs = [f"{title} [SEP] {description}" for title, description in zip(titles, descriptions)]

    # Tokenização
    encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_length)

    # Exemplo de rótulos fictícios; substitua conforme necessário
    labels = [1] * len(chunk)

    return encodings, labels



## 6. Configuração do Treinamento


In [16]:
training_args = TrainingArguments(
    output_dir='./results',          # Diretório de saída para os resultados
    num_train_epochs=3,              # Número de épocas
    per_device_train_batch_size=16,  # Tamanho do batch
    save_steps=1000,                 # Salvar checkpoints a cada 1000 passos
    save_total_limit=2,              # Limite de dois checkpoints salvos
    logging_dir='./logs',            # Diretório de logs
)


## 7. Função de Treinamento por Chunk


In [17]:
def treinar_com_chunk(encodings, labels):
    dataset = AmazonTitlesDataset(encodings, labels)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )
    # Treina o modelo usando o chunk atual
    trainer.train()


## 8. Baixar dados o Google Drive

Vamos baixar os dados do Google Drive para acessar os arquivos que contêm os dados de treinamento e teste.

In [18]:
!mkdir -p Datasets
!cd Datasets; gdown 12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK;
!cd Datasets; unzip LF-Amazon-1.3M.raw.zip; mkdir -p LF-AmazonTitles-1.3M/raw; mv LF-Amazon-1.3M/* LF-AmazonTitles-1.3M

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=f4ed60c7-2a6b-4318-bca7-855f151c061f
To: /Users/argentino/Datasets/LF-Amazon-1.3M.raw.zip
100%|████████████████████████████████████████| 890M/890M [00:31<00:00, 27.8MB/s]
Archive:  LF-Amazon-1.3M.raw.zip
   creating: LF-Amazon-1.3M/
  inflating: LF-Amazon-1.3M/lbl.json.gz  
  inflating: LF-Amazon-1.3M/trn.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_test.txt  
  inflating: LF-Amazon-1.3M/tst.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_train.txt  


## 9. Processar e Treinar em Chunks
A função principal que faz a leitura do arquivo JSONL em chunks e realiza o fine-tuning do modelo BERT em cada chunk.

In [ ]:
# Processar o arquivo compactado trn.json.gz e realizar o fine-tuning em chunks
caminho_arquivo = 'Datasets/LF-AmazonTitles-1.3M/trn.json.gz'

for i, chunk in enumerate(ler_arquivo_em_blocos_jsonl_gz(caminho_arquivo, tamanho_bloco=10000)):
    print(f"Processando chunk {i+1}")

    encodings, labels = processar_e_tokenizar_chunk(chunk)

    # Executar treinamento no chunk atual
    treinar_com_chunk(encodings, labels)

    print(f"Treinamento com chunk {i+1} completo.")



Processando chunk 1


Step,Training Loss
500,0.003100
1000,0.000000


## 10. Salvar o Modelo Fine-Tuned
Depois de processar todos os chunks e realizar o fine-tuning do modelo, salvamos o modelo treinado.

In [ ]:
# Salvar o modelo fine-tuned e o tokenizer
model.save_pretrained('./1IADT')
tokenizer.save_pretrained('./1IADT')

print("Modelo fine-tuned salvo com sucesso.")
